In [2]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split
X=df.iloc[:,1:]
Y=df.iloc[:,0]
X=X/255
Y=Y/255
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)


In [5]:
#Using XGBoost to check accuracy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model = XGBClassifier(eval_metric='auc',min_child_weight=1,max_depth=2,seed=111)
model.fit(X_train, Y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 89.75%


/home/ashwin/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
#for multilayer perceptron we will one hot encode y
#and normalize X
X=df.iloc[:,1:]
Y=df.iloc[:,0]
X=X/255
y = pd.get_dummies(Y,prefix = '')
y.head()

,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [12]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3)

model = Sequential()
model.add(Dense(X_train.shape[1], input_dim=X_train.shape[1],kernel_initializer='normal', activation='relu'))
model.add(Dense(X_train.shape[1],kernel_initializer='normal', activation='relu'))
model.add(Dense(10,kernel_initializer='normal', activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, Y_train, epochs=10, batch_size=10)
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("The summary:")
model.summary()

Epoch 1/10
29400/29400 [==============================] - 46s 2ms/step - loss: 0.2370 - acc: 0.9280
Epoch 2/10
29400/29400 [==============================] - 45s 2ms/step - loss: 0.1186 - acc: 0.9653
Epoch 3/10
29400/29400 [==============================] - 47s 2ms/step - loss: 0.0845 - acc: 0.9748
Epoch 4/10
29400/29400 [==============================] - 46s 2ms/step - loss: 0.0683 - acc: 0.9802
Epoch 5/10
29400/29400 [==============================] - 40s 1ms/step - loss: 0.0525 - acc: 0.9845
Epoch 6/10
29400/29400 [==============================] - 40s 1ms/step - loss: 0.0503 - acc: 0.9855
Epoch 7/10
29400/29400 [==============================] - 42s 1ms/step - loss: 0.0444 - acc: 0.9886
Epoch 8/10
29400/29400 [==============================] - 44s 1ms/step - loss: 0.0468 - acc: 0.9889
Epoch 9/10
29400/29400 [==============================] - 41s 1ms/step - loss: 0.0325 - acc: 0.9917
Epoch 10/10
12600/12600 [==============================] - 1s 59us/step

acc: 97.05%
The summary:
__

In [17]:
#Lets Solve by ConvNN
X_train=X_train.values
X_test=X_test.values#converting dataframe to array to reshape them lol
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [26]:
from keras.layers import Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D



model = Sequential()

model.add(Convolution2D(32,(3,3), activation='relu', input_shape=(1,28,28),padding='same'))
model.add(Convolution2D(32,(3,3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering="th"))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
 
#Compile model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
 
# Fit model on training data
model.fit(X_train, Y_train, epochs=20,batch_size=32)
 
# Evaluate model on test data
score = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

#Summary
model.summary()

/home/ashwin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/20
29400/29400 [==============================] - 5s 185us/step - loss: 0.4327 - acc: 0.8652
Epoch 2/20
29400/29400 [==============================] - 5s 169us/step - loss: 0.1597 - acc: 0.9518
Epoch 3/20
29400/29400 [==============================] - 5s 172us/step - loss: 0.1169 - acc: 0.9640
Epoch 4/20
29400/29400 [==============================] - 5s 172us/step - loss: 0.1000 - acc: 0.9683
Epoch 5/20
29400/29400 [==============================] - 5s 178us/step - loss: 0.0840 - acc: 0.9738
Epoch 6/20
29400/29400 [==============================] - 5s 175us/step - loss: 0.0749 - acc: 0.9759
Epoch 7/20
29400/29400 [==============================] - 5s 180us/step - loss: 0.0659 - acc: 0.9794
Epoch 8/20
29400/29400 [==============================] - 5s 180us/step - loss: 0.0618 - acc: 0.9804
Epoch 9/20
29400/29400 [==============================] - 5s 181us/step - loss: 0.0572 - acc: 0.9823
Epoch 10/20
29400/29400 [==============================] - 5s 178us/step - loss: 0.0516 - a